***This notebook for final functions and executions***

In [24]:
# Imports
import ffmpeg
import os
import subprocess
import re
import pandas as pd
import numpy as np

## Combined functions

In [33]:
# adding a function to round values of a list
def round_list_values(the_list, rounding=3):
    return list(np.round(the_list, rounding))

In [34]:
def get_success_array(silence_middle, split_arr):
    i = 0
    success_array = []
    for j, silence_time in enumerate(silence_middle):
        if silence_time < split_arr[i]:
            continue
        else:
            # compare which item is closer to our suggested split time
            prev_silence = silence_middle[j - 1]
            if silence_time - split_arr[i] > split_arr[i] - prev_silence:
                if prev_silence not in success_array:
                    success_array.append(prev_silence)
                else:
                    success_array.append(silence_time)
            else:
                success_array.append(silence_time)
            # move to the next time in the split_arr
            if i < len(split_arr) - 1:
                i += 1
            else:
                break
    return round_list_values(success_array)

In [35]:
def merge_small_durations(diff_arr, min_dur, max_dur, fwd=True):
    dur_arr_mod = []
    flag_raised = False  # the flag will be raised if we merges occur
    for i, dur in enumerate(diff_arr):
        if flag_raised:
            flag_raised = False
            continue
        if (dur < min_dur) & (dur > 0):
            # if dur < min_dur:
            if i + 1 < len(diff_arr):
                next_diff = diff_arr[i + 1]
                if dur + next_diff < max_dur:
                    if fwd: dur_arr_mod.append(0)
                    dur_arr_mod.append(dur + next_diff)
                    if not fwd: dur_arr_mod.append(0)
                    flag_raised = True
            else:
                dur_arr_mod.append(dur)
        else:
            dur_arr_mod.append(dur)
    return round_list_values(dur_arr_mod)

In [39]:
def split_audio_file(
    src,
    result,
    split_interval,
    min_duration,
    max_duration,
):
    split_cmd = f"ffmpeg -i {src} -af silencedetect=n=-20dB:d=0.25 -f null - "
    out = subprocess.Popen(split_cmd,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.STDOUT)
    stdout, stderr = out.communicate()
    str_out = str(stdout)
    # FInding all and converting to float in one step
    s_start = map(float, re.findall(r'silence_start: (\d+.\d+)', str_out))
    s_end = map(float, re.findall(r'silence_end: (\d+.\d+)', str_out))
    s_duration = map(float, re.findall(r'silence_duration: (\d+.\d+)',
                                       str_out))

    # zipping all together
    results = list(zip(s_start, s_end, s_duration))
    results_df = pd.DataFrame(
        results, columns=['silence_start', 'silence_end', 'silence_duration'])
    results_df[
        'silence_mid'] = results_df.silence_start + results_df.silence_duration / 2
    end_time = float(ffmpeg.probe(src)['format']['duration'])
    split_arr = np.arange(split_interval, int(end_time), split_interval)
    silence_mid = results_df.silence_mid.to_numpy()
    success_array = get_success_array(silence_mid, split_arr)
    dur_array = [0] + success_array + [end_time]
    difference_array = round_list_values(np.diff(np.array(dur_array)))
    fwd_arr = merge_small_durations(difference_array, min_duration,
                                    max_duration)
    bwd_arr = merge_small_durations(fwd_arr[::-1],
                                    min_duration,
                                    max_duration,
                                    fwd=False)[::-1]
    # stage 1
    final_stops = [x for i, x in enumerate(dur_array[1:]) if bwd_arr[i] != 0]
    # stage 2
    final_stops = final_stops[:-1]
    final_durations = round_list_values(
        np.diff(np.array([0] + final_stops + [end_time])))
    out_times = ",".join(str(e) for e in final_stops)
    slice_cmd = f"ffmpeg -i {src} -vn -c copy -f segment -segment_times "
    slice_cmd += f"{out_times} {result}"
    execute = os.system(slice_cmd)
    if execute == 0:
        print(f"The {src} file was split successfuly")
    else:
        print(f"Error during split of {src} file")
    pass

In [15]:
split_interval, min_duration, max_duration = 180, 30, 240
for file_number in range(1, 5):
    src = f"X:/Music/RightlyGuidedCalifate/RGC{file_number:03d}_.mp3"
    result = f"X:/Music/RightlyGuidedCalifate/Outputs/RGC{file_number:03d}_%d.mp3"
    print(file_number, end=". ")
    split_audio_file(src, result, split_interval, min_duration, max_duration)

1. The X:/Music/RightlyGuidedCalifate/RGC001_.mp3 file was split successfuly
2. The X:/Music/RightlyGuidedCalifate/RGC002_.mp3 file was split successfuly
3. The X:/Music/RightlyGuidedCalifate/RGC003_.mp3 file was split successfuly
4. The X:/Music/RightlyGuidedCalifate/RGC004_.mp3 file was split successfuly


In [ ]:
split_interval, min_duration, max_duration = 180, 30, 240
folder = "X:/Music/RightlyGuidedCalifate/"
for file_number in range(1, 108): # the last file is #107
    src = f"{folder}RGC{file_number:03d}_.mp3"
    result = f"{folder}Outputs/RGC{file_number:03d}_%d.mp3"
    print(file_number, end=". ")
    split_audio_file(src, result, split_interval, min_duration, max_duration)

**I want to place each 7 files in a folder**


In [29]:
file_number = 4


def find_folder_name(file_number):
    folder_number = (file_number-1) // 7 + 1
    return f"Group_{folder_number:02d}"

In [30]:
for i in (1, 2, 7, 8, 12, 13, 14, 15, 70):
    print(i, find_folder_name(i), end=">>")

1 Group_01>>2 Group_01>>7 Group_01>>8 Group_02>>12 Group_02>>13 Group_02>>14 Group_02>>15 Group_03>>70 Group_10>>

**To create the folder if not exists**
See here:

https://appdividend.com/2021/07/03/how-to-create-directory-if-not-exist-in-python/

In [27]:
def get_output_folder(source_output, file_number):
    folder_name = find_folder_name(file_number)
    folder_path = os.path.join(source_output, folder_name)
    exists = os.path.exists(folder_path)
    if not exists:
        os.makedirs(folder_path)
    return folder_path

In [31]:
for i in (1, 2, 7, 8, 12, 13, 14, 15, 70):
    print(i, get_output_folder("X:/Music/RightlyGuidedCalifate/Outputs/", i))

1 X:/Music/RightlyGuidedCalifate/Outputs/Group_01
2 X:/Music/RightlyGuidedCalifate/Outputs/Group_01
7 X:/Music/RightlyGuidedCalifate/Outputs/Group_01
8 X:/Music/RightlyGuidedCalifate/Outputs/Group_02
12 X:/Music/RightlyGuidedCalifate/Outputs/Group_02
13 X:/Music/RightlyGuidedCalifate/Outputs/Group_02
14 X:/Music/RightlyGuidedCalifate/Outputs/Group_02
15 X:/Music/RightlyGuidedCalifate/Outputs/Group_03
70 X:/Music/RightlyGuidedCalifate/Outputs/Group_10


**Creating a function of splitting**

In [37]:
split_interval, min_duration, max_duration = 180, 30, 240
source_folder = "X:/Music/RightlyGuidedCalifate"
output_folder = os.path.join(source_folder, "output")
for file_number in range(1, 5):
    src = os.path.join(source_folder, f"RGC{file_number:03d}_.mp3")
    result = os.path.join(get_output_folder(output_folder, file_number),
                          f"RGC{file_number:03d}_%d.mp3")
    print(file_number, end=". ")
    split_audio_file(src, result, split_interval, min_duration, max_duration)

1. The X:/Music/RightlyGuidedCalifate\RGC001_.mp3 file was split successfuly
2. The X:/Music/RightlyGuidedCalifate\RGC002_.mp3 file was split successfuly
3. The X:/Music/RightlyGuidedCalifate\RGC003_.mp3 file was split successfuly
4. The X:/Music/RightlyGuidedCalifate\RGC004_.mp3 file was split successfuly


In [38]:
split_interval, min_duration, max_duration = 180, 30, 240
source_folder = "X:/Music/RightlyGuidedCalifate"
output_folder = os.path.join(source_folder, "output")
for file_number in range(5, 113): #From 5 to 112
    src = os.path.join(source_folder, f"RGC{file_number:03d}_.mp3")
    result = os.path.join(get_output_folder(output_folder, file_number),
                          f"RGC{file_number:03d}_%d.mp3")
    print(file_number, end=". ")
    split_audio_file(src, result, split_interval, min_duration, max_duration)

5. The X:/Music/RightlyGuidedCalifate\RGC005_.mp3 file was split successfuly
6. The X:/Music/RightlyGuidedCalifate\RGC006_.mp3 file was split successfuly
7. The X:/Music/RightlyGuidedCalifate\RGC007_.mp3 file was split successfuly
8. The X:/Music/RightlyGuidedCalifate\RGC008_.mp3 file was split successfuly
9. The X:/Music/RightlyGuidedCalifate\RGC009_.mp3 file was split successfuly
10. Error during split of X:/Music/RightlyGuidedCalifate\RGC010_.mp3 file
11. 

IndexError: list index out of range

In [40]:
split_interval, min_duration, max_duration = 180, 30, 240
source_folder = "X:/Music/RightlyGuidedCalifate"
output_folder = os.path.join(source_folder, "output")
for file_number in range(10, 113): #From 5 to 112
    src = os.path.join(source_folder, f"RGC{file_number:03d}_.mp3")
    result = os.path.join(get_output_folder(output_folder, file_number),
                          f"RGC{file_number:03d}_%d.mp3")
    print(file_number, end=". ")
    split_audio_file(src, result, split_interval, min_duration, max_duration)

10. The X:/Music/RightlyGuidedCalifate\RGC010_.mp3 file was split successfuly
11. The X:/Music/RightlyGuidedCalifate\RGC011_.mp3 file was split successfuly
12. The X:/Music/RightlyGuidedCalifate\RGC012_.mp3 file was split successfuly
13. The X:/Music/RightlyGuidedCalifate\RGC013_.mp3 file was split successfuly
14. The X:/Music/RightlyGuidedCalifate\RGC014_.mp3 file was split successfuly
15. The X:/Music/RightlyGuidedCalifate\RGC015_.mp3 file was split successfuly
16. The X:/Music/RightlyGuidedCalifate\RGC016_.mp3 file was split successfuly
17. The X:/Music/RightlyGuidedCalifate\RGC017_.mp3 file was split successfuly
18. The X:/Music/RightlyGuidedCalifate\RGC018_.mp3 file was split successfuly
19. The X:/Music/RightlyGuidedCalifate\RGC019_.mp3 file was split successfuly
20. The X:/Music/RightlyGuidedCalifate\RGC020_.mp3 file was split successfuly
21. The X:/Music/RightlyGuidedCalifate\RGC021_.mp3 file was split successfuly
22. The X:/Music/RightlyGuidedCalifate\RGC022_.mp3 file was spli